In [ ]:
import warnings
import pickle

import pandas as pd
import numpy as np

from tqdm import tqdm 

warnings.filterwarnings('ignore')
PATH_TO_DATA = 'data/' 

In [ ]:
train_df = pd.read_csv(PATH_TO_DATA +'train_sessions_400users.csv', index_col='session_id', parse_dates=time_cols)

with open(PATH_TO_DATA + 'site_dic.pkl', 'rb') as site_file:
     sites_dict = pickle.load(site_file)
        
id_sites_dict = {v: k for k, v in sites_dict.items()}

train_df.shape

In [ ]:
def rolling_window(a, window, roll_step = 3):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    arr = np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
    return np.array([v for idx, v in enumerate(arr) if idx % roll_step == 0 ])

In [ ]:
def rolling(d, t):
    if len(d) < 10:
        return np.array([d + [0] * (10 - len(d))]), np.array([t + [np.NaN] * (10 - len(d))])
    else:
        return rolling_window(np.array(d), 10), rolling_window(np.array(t), 10)

The idea is slightly increase the size of the train dataset by simply rolling user's sessions. It allows me to get new sessions which aren't equal to existing ones.

In [ ]:
grouped = train_df.groupby(by='user_id')
new_data = pd.DataFrame([], columns=site_cols + time_cols + ["user_id"])

for g in tqdm(grouped.groups.keys()):
    vals = grouped.get_group(g) 
    vals = vals.loc[:, vals.columns != 'user_id'].values
    vals = vals.reshape(1, vals.shape[0] * vals.shape[1])[0]
    
    values = [v for idx, v in enumerate(vals) if idx % 2 == 0 ]
    index = [v for idx, v in enumerate(vals) if idx % 2 == 1 ]
    
    s = pd.Series(values, index=index).sort_index()
    s.dropna(inplace=True)
    
    sites_row = []
    times_row = []

    first_idx = 0

    while first_idx < s.shape[0]:
        v_tmp = []
        t_tmp = []
        curr_idx = first_idx

        while curr_idx < s.shape[0] and (s.index[curr_idx] - s.index[first_idx]).seconds < 1800:
            v_tmp.append(s.values[curr_idx])
            t_tmp.append(s.index[curr_idx])

            curr_idx += 1

        first_idx = curr_idx

        sites_row.append(v_tmp)
        times_row.append(t_tmp)
        
    t_values_row = []
    t_times_row = []

    for v, t in zip(sites_row, times_row):
        new_vals, new_times = rolling(v, t)
        
        t_values_row.extend(new_vals)
        t_times_row.extend(new_times)
            
            
    t_data = [v.tolist() + t.tolist() for v, t in zip(t_values_row, t_times_row) ]
    row = pd.DataFrame(t_data, columns=site_cols + time_cols)
    row['user_id'] = [g] * row.shape[0]
    
    new_data = pd.concat([new_data, row], ignore_index=True)

In [ ]:
new_data.sort_values(by=['time1'], inplace=True, ascending=True)
new_data.reset_index(inplace=True, drop=True)

new_data.to_csv(PATH_TO_DATA +'rolled.csv')

new_data.shape